In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
os.environ['BRUNEL_CONFIG'] = "locjavascript=/data/jupyter2/static-file-content-delivery-network/nbextensions/brunel_ext"
import brunel
# Brunel will be used to visualize data later on

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = application_1576188780064_0008


In [2]:
!pip install --user brunel

  Using cached https://files.pythonhosted.org/packages/04/de/2d314a921ef4c20b283e1de94e0780273678caac901564df06b948e4ba9b/py4j-0.10.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/00/17/c3e3dd337cc601b47c599441786fd4e888b587a2a512d38ac752274fd0bc/ipython-7.10.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/65/e0/eb35e762802015cab1ccee04e8a277b03f1d8e53da3ec3106882ec42558b/Jinja2-2.10.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/e0/a1b39cdcb2c391f087a1538bc8a6d62a82d0439693192aef541d7b123769/pandas-0.25.3-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/7f/1f/e145dd467dc9b0e6f1e64232c03119498dfec497e383f1e8be9f83eaa97e/prompt_toolkit-3.0.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ca/ab/872a23e29cec3cf2594af7e857f18b687ad21039c1f9b922fac5b9b142d5/traitlets-4.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.or

In [4]:

import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': 'Grr-Oyo22GjKDnoB7eYhAIIsr9icC7i8htPkWOVczNvy',
    'service_id': 'iam-ServiceId-91b36dec-75c8-4e99-a58d-aed987e7b830',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token'}

configuration_name = 'os_d4d02dc6ab484d629e81fbdba78adf13_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
hotel = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('Hotel_Reviewss.csv', 'bbb-donotdelete-pr-zvfu5f9w3y398c'))
hotel.take(5)


[Row(Hotel_Address=' s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands', Additional_Number_of_Scoring='194', Review_Date='08/03/2017', Average_Score='7.7', Hotel_Name='Hotel Arena', Reviewer_Nationality=' Russia ', Negative_Review=' I am so angry that i made this post available via all possible sites i use when planing my trips so no one will make the mistake of booking this place I made my booking via booking com We stayed for 6 nights in this hotel from 11 to 17 July Upon arrival we were placed in a small room on the 2nd floor of the hotel It turned out that this was not the room we booked I had specially reserved the 2 level duplex room so that we would have a big windows and high ceilings The room itself was ok if you don t mind the broken window that can not be closed hello rain and a mini fridge that contained some sort of a bio weapon at least i guessed so by the smell of it I intimately asked to change the room and after explaining 2 times that i booked a duplex btw it c

In [5]:
hotel.count()

511950

In [6]:
hotel.printSchema()

root
 |-- Hotel_Address: string (nullable = true)
 |-- Additional_Number_of_Scoring: string (nullable = true)
 |-- Review_Date: string (nullable = true)
 |-- Average_Score: string (nullable = true)
 |-- Hotel_Name: string (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Negative_Review: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: string (nullable = true)
 |-- Total_Number_of_Reviews: string (nullable = true)
 |-- Positive_Review: string (nullable = true)
 |-- Review_Total_Positive_Word_Counts: string (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: string (nullable = true)
 |-- Reviewer_Score: string (nullable = true)
 |-- Tags: string (nullable = true)
 |-- days_since_review: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)



In [7]:

hotel.createOrReplaceTempView("hot")

In [8]:
prueba = spark.sql("select Reviewer_Nationality as Nacionalidad, count(Reviewer_Nationality) as Cuenta from hot where Hotel_Name = 'Hotel Arena' group by Reviewer_Nationality order by Reviewer_Nationality")

In [9]:
prueba.show(5)

+------------------+------+
|      Nacionalidad|Cuenta|
+------------------+------+
| Abkhazia Georgia |     1|
|        Argentina |     1|
|        Australia |    13|
|          Austria |     2|
|          Belgium |     5|
+------------------+------+
only showing top 5 rows



In [10]:
custom_frame = prueba.toPandas()
custom_frame.head(4)

,Nacionalidad,Cuenta
0,Abkhazia Georgia,1
1,Argentina,1
2,Australia,13
3,Austria,2


In [11]:
# Bubble de reviewsl hotel arena
%brunel data('custom_frame') bubble size(Cuenta) sort(Cuenta) color(Nacionalidad) label(Nacionalidad, Cuenta) legends(none) tooltip(Nacionalidad)

<IPython.core.display.Javascript object>

In [12]:
prueba2 = spark.sql("select Reviewer_Nationality as Nacionalidad, count(Reviewer_Nationality) as Cuenta from hot group by Reviewer_Nationality order by Reviewer_Nationality")

In [13]:
custom_frame = prueba2.toPandas()

In [14]:
# Bubble de reviews por nacionalidades
%brunel data('custom_frame') bubble size(Cuenta) sort(Cuenta) color(Nacionalidad) label(Nacionalidad, Cuenta) legends(none) tooltip(Nacionalidad)

<IPython.core.display.Javascript object>

In [41]:
#Promedio de calificacion por nacionalidad de Hotel araena
prueba3 = spark.sql("select Reviewer_Nationality, count(Reviewer_Score) as Visitantes, sum(Reviewer_Score)/count(Reviewer_Score) as Promedio from hot where Hotel_Name in (select Hotel_Name from hot Where Hotel_Name ='Hotel Arena') group by Reviewer_Nationality order by Visitantes desc")

In [42]:
custom_frame = prueba3.toPandas()

In [43]:
custom_frame.head(4)

,Reviewer_Nationality,Visitantes,Promedio
0,United Kingdom,208,8.086538
1,Ireland,31,7.877419
2,United States of America,20,7.505000
3,Netherlands,15,7.600000


In [44]:
%brunel data('custom_frame') bubble size(Visitantes) sort(Promedio) color(Reviewer_Nationality) label(Reviewer_Nationality, Promedio) legends(none) tooltip(Reviewer_Nationality, Promedio, Visitantes)

<IPython.core.display.Javascript object>

In [205]:
#Promedio de calificacion por nacionalidad de Hotel araena
prueba4 = spark.sql("select Reviewer_Nationality as Nacionalidad, count(Reviewer_Score) as Visitantes, sum(Reviewer_Score)/count(Reviewer_Score) as Promedio from hot group by Reviewer_Nationality having Visitantes > 500 order by Promedio desc limit 20")

In [206]:
custom_frame = prueba4.toPandas()

In [191]:
custom_frame.count()

Reviewer_Nationality    1
Visitantes              1
Promedio                1
dtype: int64

In [204]:
prueba4.show(3000)

+--------------------+----------+------------------+
|        Nacionalidad|Visitantes|          Promedio|
+--------------------+----------+------------------+
| United States of...|     35196| 8.788353790203809|
|             Israel |      6536| 8.695104039167605|
|        New Zealand |      3222| 8.654407200496609|
|          Australia |     21540|   8.5914345403902|
|             Mexico |       583| 8.571526586620921|
|             Canada |      7813|  8.54573147318561|
|            Hungary |      2114| 8.514995269631044|
|     United Kingdom |    244457| 8.486110440691363|
|              Malta |      1662|  8.46919374247896|
|            Ireland |     14746| 8.463156110131552|
|            Iceland |       930|  8.46172043010753|
|              China |      3379| 8.445457235868611|
|             Brazil |      1850| 8.438486486486507|
|        Philippines |      1069| 8.428811973807305|
|          Argentina |       511|  8.41878669275929|
|           Bulgaria |      1287| 8.4142968142

In [208]:
%brunel data('custom_frame') bubble size(Visitantes) sort(Promedio) color(Nacionalidad) label(Nacionalidad, Promedio) legends(none) tooltip(Nacionalidad, Promedio, Visitantes)

<IPython.core.display.Javascript object>

In [50]:
#Promedio de calificacion General
prueba5 = spark.sql("select count(Reviewer_Score) as Visitantes, sum(Reviewer_Score)/count(Reviewer_Score) as Promedio from hot order by Visitantes desc")

In [51]:
custom_frame = prueba5.toPandas()

In [52]:
prueba5.head(5)

[Row(Visitantes=511950, Promedio=8.395985740801539)]

In [55]:
#Mapeos
#Este es de Amsterdam
prueba6 = spark.sql("select Distinct Hotel_Name, lat, lng, Average_Score from hot where Hotel_Name in (select distinct Hotel_Name from hot order by Hotel_Name asc) and Hotel_Address like '%Netherlands%' order by Hotel_Name asc ")

In [57]:
mapPan=prueba6.toPandas()
%brunel map('prueba') + data('mapPan') x(lng) y(lat) color(Average_Score) tooltip(Hotel_Name, Average_Score)

<IPython.core.display.Javascript object>

In [58]:
#Mapeos
#Este es de UK
prueba7 = spark.sql("select Distinct Hotel_Name, lat, lng, Average_Score from hot where Hotel_Name in (select distinct Hotel_Name from hot order by Hotel_Name asc) and Hotel_Address like '%United Kingdom%' order by Hotel_Name asc ")
mapPan=prueba7.toPandas()
%brunel map('prueba') + data('mapPan') x(lng) y(lat) color(Average_Score) tooltip(Hotel_Name, Average_Score)

<IPython.core.display.Javascript object>

In [59]:
#Mapeos
#Este es de Francia
prueba8 = spark.sql("select Distinct Hotel_Name, lat, lng, Average_Score from hot where Hotel_Name in (select distinct Hotel_Name from hot order by Hotel_Name asc) and Hotel_Address like '%Paris France%' order by Hotel_Name asc ")
mapPan=prueba8.toPandas()
%brunel map('prueba') + data('mapPan') x(lng) y(lat) color(Average_Score) tooltip(Hotel_Name, Average_Score)

<IPython.core.display.Javascript object>

In [60]:
#Mapeos
#Este es de Barcelona
prueba9 = spark.sql("select Distinct Hotel_Name, lat, lng, Average_Score from hot where Hotel_Name in (select distinct Hotel_Name from hot order by Hotel_Name asc) and Hotel_Address like '%Barcelona Spain%' order by Hotel_Name asc ")
mapPan=prueba9.toPandas()
%brunel map('prueba') + data('mapPan') x(lng) y(lat) color(Average_Score) tooltip(Hotel_Name, Average_Score)

<IPython.core.display.Javascript object>

In [61]:
#Mapeos
#Este es de Vienna
prueba9 = spark.sql("select Distinct Hotel_Name, lat, lng, Average_Score from hot where Hotel_Name in (select distinct Hotel_Name from hot order by Hotel_Name asc) and Hotel_Address like '%Vienna Austria%' order by Hotel_Name asc ")
mapPan=prueba9.toPandas()
%brunel map('prueba') + data('mapPan') x(lng) y(lat) color(Average_Score) tooltip(Hotel_Name, Average_Score)

<IPython.core.display.Javascript object>

In [62]:
#Mapeos
#Este es de Milan
prueba10 = spark.sql("select Distinct Hotel_Name, lat, lng, Average_Score from hot where Hotel_Name in (select distinct Hotel_Name from hot order by Hotel_Name asc) and Hotel_Address like '%Milan Italy%' order by Hotel_Name asc ")
mapPan=prueba10.toPandas()
%brunel map('prueba') + data('mapPan') x(lng) y(lat) color(Average_Score) tooltip(Hotel_Name, Average_Score)

<IPython.core.display.Javascript object>

In [69]:
#Mejores Hoteles
prueba11 = spark.sql("select Distinct Hotel_Name, Average_Score from hot order by Average_Score desc limit 6")
custom_frame=prueba11.toPandas()

In [71]:
custom_frame.head(6)

,Hotel_Name,Average_Score
0,Ritz Paris,9.8
1,Haymarket Hotel,9.6
2,H10 Casa Mimosa 4 Sup,9.6
3,H tel de La Tamise Esprit de France,9.6
4,Hotel Casa Camper,9.6
5,41,9.6


In [72]:
#Peores Hoteles
prueba11 = spark.sql("select Distinct Hotel_Name, Average_Score from hot order by Average_Score asc limit 6")
custom_frame=prueba11.toPandas()
custom_frame.head(6)

,Hotel_Name,Average_Score
0,Hotel Liberty,5.2
1,Savoy Hotel Amsterdam,6.4
2,Hotel Cavendish,6.4
3,Best Western Maitrise Hotel Edgware Road,6.6
4,The Tophams Hotel,6.6
5,Commodore Hotel,6.7


In [159]:
#A Dónde van los de UK
origen = 'Mexico'
destino = 'Netherlands'
prueba11 = spark.sql("select count(Hotel_Name) as Cuenta from hot where Reviewer_Nationality like '%" +origen+"%' and Hotel_Address like '%" +destino+"%' ")
prueba12 = spark.sql("select count(Hotel_Name) as Cuenta from hot where Reviewer_Nationality like '%" +origen+"%'")
cf1 = prueba11.toPandas()
cf2 = prueba12.toPandas()
viajeros_esp = cf1.iloc[0]['Cuenta']
viajeros_tot = cf2.iloc[0]['Cuenta']
porc_viaj_dePais= viajeros_esp/viajeros_tot
print(porc_viaj_dePais)

0.14751286449399656


In [160]:
origen = 'Mexico'
destino = 'Barcelona Spain'
prueba11 = spark.sql("select count(Hotel_Name) as Cuenta from hot where Reviewer_Nationality like '%" +origen+"%' and Hotel_Address like '%" +destino+"%' ")
prueba12 = spark.sql("select count(Hotel_Name) as Cuenta from hot where Reviewer_Nationality like '%" +origen+"%'")
cf1 = prueba11.toPandas()
cf2 = prueba12.toPandas()
viajeros_esp = cf1.iloc[0]['Cuenta']
viajeros_tot = cf2.iloc[0]['Cuenta']
porc_viaj_dePais= viajeros_esp/viajeros_tot
print(porc_viaj_dePais)

0.10291595197255575


In [161]:
origen = 'Mexico'
destino = 'Paris France'
prueba11 = spark.sql("select count(Hotel_Name) as Cuenta from hot where Reviewer_Nationality like '%" +origen+"%' and Hotel_Address like '%" +destino+"%' ")
prueba12 = spark.sql("select count(Hotel_Name) as Cuenta from hot where Reviewer_Nationality like '%" +origen+"%'")
cf1 = prueba11.toPandas()
cf2 = prueba12.toPandas()
viajeros_esp = cf1.iloc[0]['Cuenta']
viajeros_tot = cf2.iloc[0]['Cuenta']
porc_viaj_dePais= viajeros_esp/viajeros_tot
print(porc_viaj_dePais)

0.27101200686106347


In [162]:
origen = 'Mexico'
destino = 'United Kingdom'
prueba11 = spark.sql("select count(Hotel_Name) as Cuenta from hot where Reviewer_Nationality like '%" +origen+"%' and Hotel_Address like '%" +destino+"%' ")
prueba12 = spark.sql("select count(Hotel_Name) as Cuenta from hot where Reviewer_Nationality like '%" +origen+"%'")
cf1 = prueba11.toPandas()
cf2 = prueba12.toPandas()
viajeros_esp = cf1.iloc[0]['Cuenta']
viajeros_tot = cf2.iloc[0]['Cuenta']
porc_viaj_dePais= viajeros_esp/viajeros_tot
print(porc_viaj_dePais)

0.27101200686106347


In [163]:
origen = 'Mexico'
destino = 'Milan Italy'
prueba11 = spark.sql("select count(Hotel_Name) as Cuenta from hot where Reviewer_Nationality like '%" +origen+"%' and Hotel_Address like '%" +destino+"%' ")
prueba12 = spark.sql("select count(Hotel_Name) as Cuenta from hot where Reviewer_Nationality like '%" +origen+"%'")
cf1 = prueba11.toPandas()
cf2 = prueba12.toPandas()
viajeros_esp = cf1.iloc[0]['Cuenta']
viajeros_tot = cf2.iloc[0]['Cuenta']
porc_viaj_dePais= viajeros_esp/viajeros_tot
print(porc_viaj_dePais)

0.1097770154373928


In [164]:
origen = 'Mexico'
destino = 'Vienna Austria'
prueba11 = spark.sql("select count(Hotel_Name) as Cuenta from hot where Reviewer_Nationality like '%" +origen+"%' and Hotel_Address like '%" +destino+"%' ")
prueba12 = spark.sql("select count(Hotel_Name) as Cuenta from hot where Reviewer_Nationality like '%" +origen+"%'")
cf1 = prueba11.toPandas()
cf2 = prueba12.toPandas()
viajeros_esp = cf1.iloc[0]['Cuenta']
viajeros_tot = cf2.iloc[0]['Cuenta']
porc_viaj_dePais= viajeros_esp/viajeros_tot
print(porc_viaj_dePais)

0.09777015437392796


In [187]:
#Mejores Hoteles
origen = 'Mexico'
prueba12 = spark.sql("select Hotel_Name, count(Reviewer_Score) as Visitas ,sum(cast(Reviewer_Score as DOUBLE))/cast(count(Reviewer_Score)as DOUBLE) as Promedio from hot where Reviewer_Nationality like '%" +origen+"%' GROUP BY Hotel_Name order by Visitas desc")
prueba12.show(1000)

+--------------------+-------+------------------+
|          Hotel_Name|Visitas|          Promedio|
+--------------------+-------+------------------+
|Park Plaza Westmi...|      8|            9.3375|
|NH Carlton Amsterdam|      7|7.1571428571428575|
|    Room Mate Aitana|      6| 9.383333333333335|
|Sofitel London St...|      4|               9.8|
|         Glam Milano|      4|              8.85|
|Golden Tulip Amst...|      4|             8.025|
|NH Collection Mil...|      4|              8.45|
|Austria Trend Hot...|      4|               9.6|
|Hyatt Regency Par...|      3| 7.933333333333334|
|Park Grand Paddin...|      3| 8.200000000000001|
|Best Western Blue...|      3| 9.433333333333334|
|Holiday Inn Paris...|      3| 8.333333333333334|
|    Ayre Hotel Caspe|      3|               8.9|
|            1K Hotel|      3|               9.6|
|  citizenM Amsterdam|      3| 9.866666666666667|
|Le Pavillon de la...|      3|7.8999999999999995|
|The Square Milano...|      3|               9.6|


,Hotel_Name,Visitas,Promedio
0,Park Plaza Westminster Bridge London,8,9.337500
1,NH Carlton Amsterdam,7,7.157143
2,Room Mate Aitana,6,9.383333
3,Sofitel London St James,4,9.800000
4,Golden Tulip Amsterdam West,4,8.025000
...,...,...,...
95,Carlyle Brera Hotel,2,9.200000
96,Maison Albar H tel Paris Champs Elys es ex Mac...,2,9.800000
97,Petit Palace Museum,2,8.350000
98,H tel Duo,2,9.400000
